In [12]:
import pandas as pd
import numpy as np

In [18]:
# loans_url = 'https://websdk-assets.s3.ap-south-1.amazonaws.com/public/txns+(13).csv'
# repayments_url = 'https://websdk-assets.s3.ap-south-1.amazonaws.com/public/repayments+(2).csv'
# loans_df = pd.read_csv(loans_url)
# repayments_df = pd.read_csv(repayments_url)

#with paths
# Load the datasets
loans_df = pd.read_csv('C:/trade/txns+(13).csv')
repayments_df = pd.read_csv('C:/trade/repayments+(2).csv')
# loans_df = pd.read_csv(loans_df)
# repayments_df = pd.read_csv(repayments_df)


In [16]:


# Converting amounts from paise to rupees
loans_df['emiPrincipal'] = loans_df['emiPrincipal'] / 100
repayments_df['amount'] = repayments_df['amount'] / 100

grouping_repayments = repayments_df.groupby('transactionId')['amount'].sum().reset_index()
loans_df['totalPrincipal'] = loans_df['tenure'] * loans_df['emiPrincipal']

# Merging on transactionId
merged_df = pd.merge(loans_df, grouping_repayments, on='transactionId', how='left')
merged_df['outstandingAmount'] = merged_df['totalPrincipal'] - merged_df['amount'].fillna(0)

output_df = merged_df[['transactionId', 'outstandingAmount']]

print(output_df.head())


   transactionId  outstandingAmount
0         704663            -599.50
1        1403240            -333.00
2        1961345           -2386.34
3        1682956           -2799.34
4        2392323            1405.68


In [6]:

# Converting  repaymentDay timezone 
repayments_df['repaymentDay'] = pd.to_datetime(repayments_df['repaymentDay']).dt.tz_localize(None)

def calculate_aum(as_of_date):
    filtered_repayments = repayments_df[repayments_df['repaymentDay'] <= as_of_date]
    repayments_grouped = filtered_repayments.groupby('transactionId')['amount'].sum().reset_index()

    # Calculating total principal to be paid per loan
    loans_df['totalPrincipal'] = loans_df['tenure'] * loans_df['emiPrincipal']

    merged_df = pd.merge(loans_df, repayments_grouped, on='transactionId', how='left')

    merged_df['outstandingAmount'] = merged_df['totalPrincipal'] - merged_df['amount'].fillna(0)

    aum = merged_df['outstandingAmount'].sum()

    return aum

#user input for date
date_input = input("Enter the date (YYYY-MM-DD): ")
as_of_date = pd.to_datetime(date_input)

# Calculate AUM
aum_as_of_date = calculate_aum(as_of_date)
print(f"AUM as of {date_input}: {aum_as_of_date} rupees")

Enter the date (YYYY-MM-DD):  2024-08-15


AUM as of 2024-08-15: -860593.27 rupees


In [12]:
loans_df['emiPrincipal'] = loans_df['emiPrincipal'] / 100
repayments_df['amount'] = repayments_df['amount'] / 100

# Convert day and repaymentDay to datetime
loans_df['day'] = pd.to_datetime(loans_df['day']).dt.tz_localize(None)
repayments_df['repaymentDay'] = pd.to_datetime(repayments_df['repaymentDay']).dt.tz_localize(None)

# Calculating the due dates for each EMI
loans_df['due_date'] = loans_df['day'] + pd.to_timedelta(30, unit='d')

repayments_merged = pd.merge(repayments_df, loans_df[['transactionId', 'due_date', 'emiPrincipal', 'day']], on='transactionId', how='left')

repayments_merged['dpd'] = (repayments_merged['repaymentDay'] - repayments_merged['due_date']).dt.days

conditions = [
    (repayments_merged['dpd'] < 0),
    (repayments_merged['dpd'] >= 0) & (repayments_merged['dpd'] < 30),
    (repayments_merged['dpd'] >= 30) & (repayments_merged['dpd'] < 60),
    (repayments_merged['dpd'] >= 60) & (repayments_merged['dpd'] < 90),
    (repayments_merged['dpd'] >= 90)
]
choices = ['amount_not_due', 'amount_dpd_0_30', 'amount_dpd_30_60', 'amount_dpd_60_90', 'amount_dpd_90_above']
repayments_merged['dpd_bucket'] = np.select(conditions, choices, default='amount_not_due')

repayments_merged['month'] = repayments_merged['day'].dt.to_period('M')
dpd_agg = repayments_merged.pivot_table(index='month', columns='dpd_bucket', values='amount', aggfunc='sum', fill_value=0).reset_index()

dpd_agg = dpd_agg.reindex(columns=['month', 'amount_not_due', 'amount_dpd_0_30', 'amount_dpd_30_60', 'amount_dpd_60_90', 'amount_dpd_90_above'], fill_value=0)

print(dpd_agg)

dpd_bucket    month  amount_not_due  amount_dpd_0_30  amount_dpd_30_60  \
0           2022-02          9.3300           9.3300            9.3300   
1           2022-03         26.6667          26.6667           26.6666   
2           2022-04         48.8301           5.6600            0.0000   
3           2022-05         54.8301           2.9933           10.0000   
4           2022-06        113.3334           0.0000            0.0000   
5           2022-07         46.8060          11.1202            3.3300   
6           2022-08        180.6052           6.3266            0.0000   
7           2022-09        114.2839         119.7798           58.1464   
8           2022-10        131.0672          76.2699           33.8030   
9           2022-11         61.8268          40.8766           32.2133   
10          2023-01         42.4835          52.6532            0.0000   
11          2023-02        125.2151          64.7905           26.8466   
12          2023-03        187.6777   